# Energy Forecasting

What are you actually trying to predict?
1.  What am I going to use for the rest of the year, given where I am now and the rough trend from last year?
2. What does the equate to in terms of cost, assuming that the tariff remains the same for the coming year?
3. Is there a trigger point where a particular average outside temperature coincides with the increased use of heating?

We've managed to get the SARIMA forecast working, though the validity of that model is in question because ChatGPT wrote most of the code and honestly I don't really understand the tuning factors properly.  Nonetheless, it lets us predict total use for the rest of a period of time (lets say the year).  From that you can calculate the rough remaining spend based on the current tariff.

This is stll an interesting problem; you need to keep a record of the previous tariffs and what you've used during that period.

It'd be good to be able to generate updated predictions each time new data is added, and store the old predictions so you can visualise the change as new data is added.

In [38]:
import pandas as pd
# import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# import matplotlib.pyplot as plt

---
## 1. Totals forecasting

Using an untuned SARIMA model, with a seasonality of 26 weeks, we can at least get a rough estimate of the expected meter reading at a given week.  We can use that to get a cost estimate if we just merge the use rate with the tariff table.

First, import the data table and feature engineer such that we have a reading every week, rather than the irregular frequency in the actual data table.

In [36]:
# Import the data from file and exclude the columns you don't need.
readings = pd.read_excel("Energy Usage.xlsx", 
                   sheet_name="Data", 
                   skiprows=9,
                   usecols=[0,2,3],
                   names=['Date', 'Elec', 'Gas'],
                   dtype={'Gas': 'float32', 'Elec': 'int32'})

# Cut off everything after 2021 - started taking readings weekly in 2022.
readings = readings.loc[readings['Date'] > "2022"].reset_index(drop=True)

# Create date range from 31/3/2022 until today
df_range = pd.date_range(start=readings['Date'][0], end=readings['Date'][len(readings)-1], freq='D').to_frame().reset_index(drop=True)
df_range.columns = ["Date"]

# # Merge the meter readings into the df_range table
readings = pd.merge(left=df_range, right=readings, how='outer', left_on='Date', right_on='Date')

# Interpolate the data between entries
readings['Gas Interpolation'] = readings['Gas'].interpolate(method='linear')
readings['Elec Interpolation'] = readings['Elec'].interpolate(method='linear')
readings = readings[['Date','Gas Interpolation', 'Elec Interpolation']]
readings.set_index('Date', inplace=True) 
readings.rename(columns={'Gas Interpolation':'Gas', 'Elec Interpolation': 'Elec'}, inplace=True)
readings = readings.iloc[::7,:]

readings.head()

,Gas,Elec
Date,,
2022-03-31,3195.000000,9692.0
2022-04-07,3207.000000,9764.0
2022-04-14,3215.438965,9836.0
2022-04-21,3220.407959,9889.0
2022-04-28,3226.030029,9950.0


Now use the SARIMAX model with a seasonality of 26 (for 26 weeks) to create prediction models for both Gas and Electric.

In [39]:
# Set the number of weeks into the future you want predicted for
prediction_steps = 8

# Parameters for the models - using the same for both.
p, d, q = 1, 1, 1  # Non-seasonal parameters
P, D, Q, s = 1, 1, 1, 26  # Seasonal parameters

# Fit a SARIMA model for gas
sarima_gas_model = SARIMAX(readings['Gas'], order=(p, d, q), seasonal_order=(P, D, Q, s))
sarima_gas_result = sarima_gas_model.fit()

# Make predictions on the test set for Gas
gas_predictions = sarima_gas_result.get_forecast(steps=prediction_steps)



# Fit a SARIMA model for electricity
sarima_electricity_model = SARIMAX(readings['Elec'], order=(p, d, q), seasonal_order=(P, D, Q, s))
sarima_electricity_result = sarima_electricity_model.fit()

# Make predictions on the test set for Electricity
electricity_predictions = sarima_electricity_result.get_forecast(steps=prediction_steps)

/home/jon/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-THU will be used.
  self._init_dates(dates, freq)
/home/jon/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-THU will be used.
  self._init_dates(dates, freq)
/home/jon/venv/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.31078D+00    |proj g|=  2.99239D-01

At iterate    5    f=  2.15427D+00    |proj g|=  1.52621D-01

At iterate   10    f=  1.99035D+00    |proj g|=  2.51307D-02

At iterate   15    f=  1.98906D+00    |proj g|=  4.02105D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     19     22      1     0     0   9.939D-07   1.989D+00
  F =   1.9890564300810205     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


/home/jon/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-THU will be used.
  self._init_dates(dates, freq)
/home/jon/venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-THU will be used.
  self._init_dates(dates, freq)
/home/jon/venv/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.05408D+00    |proj g|=  2.57785D-01

At iterate    5    f=  2.96751D+00    |proj g|=  2.54625D-02

At iterate   10    f=  2.93514D+00    |proj g|=  3.86060D-02

At iterate   15    f=  2.89527D+00    |proj g|=  1.08297D-02

At iterate   20    f=  2.88744D+00    |proj g|=  8.94575D-03

At iterate   25    f=  2.87152D+00    |proj g|=  4.11585D-02

At iterate   30    f=  2.86763D+00    |proj g|=  6.85299D-03

At iterate   35    f=  2.86679D+00    |proj g|=  2.14566D-03

At iterate   40    f=  2.86674D+00    |proj g|=  2.83162D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg 

With that complete, we can create a dataframe of the predictions against date.

In [40]:
predictions = pd.DataFrame([gas_predictions.predicted_mean, electricity_predictions.predicted_mean]).transpose()
predictions

,predicted_mean,predicted_mean
2023-11-23,3962.726573,14170.411501
2023-11-30,3985.005481,14239.220223
2023-12-07,4011.139642,14320.030864
2023-12-14,4050.102532,14414.044406
2023-12-21,4089.771537,14483.988497
2023-12-28,4116.827925,14534.434526
2024-01-04,4139.412062,14600.652194
2024-01-11,4165.911191,14647.011223


---
## 2.0 Matchig Tariff Data

Now that we have this, we want to ue these predictions to estimate cost for the period forecast.  Generally, I'd like to be able to have a forecast curve for the whole year plot actual readings against it to see how much we're varying from the forecast, and use the forecast to tell me how much money I need to plan to spend for the rest of the year.